In [ ]:
import numpy as np
import pandas as pd
from nilearn import surface
import nibabel as nib

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.pyplot import MultipleLocator
import seaborn as sns

from matplotlib import font_manager
font_manager.fontManager.addfont("/n02dat01/users/lchai/anaconda3/envs/Nm/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/arial.ttf")
plt.rcParams["font.sans-serif"] = "Arial" 

In [ ]:
dirc_L = f'/n02dat01/users/dyli/Atlas/metric_index_L.txt'
select_ind_L = np.loadtxt( dirc_L ).astype(int)
dirc_R = f'/n02dat01/users/dyli/Atlas/metric_index_R.txt'
select_ind_R = np.loadtxt( dirc_R ).astype(int)

# read the Yeo Atlas
atlas_path = '/n02dat01/users/dyli/Atlas/Schaefer2018_400Parcels_17Networks_order.dlabel.nii'
atlas = nib.load(atlas_path)
atlas_data_ = atlas.get_fdata()
atlas_data_ = np.squeeze(atlas_data_) #(64984,)
atlas_data = np.zeros(59412)
atlas_data[0:29696] = atlas_data_[select_ind_L]
atlas_data[29696:59412] = atlas_data_[select_ind_R+32492]

# biorxiv LVs

In [ ]:
# read the basic info
dirc_L = '/n02dat01/users/dyli/Atlas/metric_index_L.txt'
select_ind_L = np.loadtxt( dirc_L ).astype(int)
dirc_R = '/n02dat01/users/dyli/Atlas/metric_index_R.txt'
select_ind_R = np.loadtxt( dirc_R ).astype(int)

atlas_data = atlas_data[0:29696]

data_L = np.load('/n02dat01/users/dyli/Grad_data/sc/MPC_Grad_results/MPC_sc_vertex_volume_100_embedding_dense_emb_L_zeros.npy')
data_R = np.load('/n02dat01/users/dyli/Grad_data/sc/MPC_Grad_results/MPC_sc_vertex_volume_100_embedding_dense_emb_R_zeros.npy')

for g in range(3):
    # read the TX gradient data
    Tx_data = nib.load(f'/n02dat01/users/dyli/Grad_data/support_code/Vogel_PLS_Tx-Space/surf/PLS_Volume_C{g+1}_Smth0mm_surf5mm_gc.func.gii')
    Tx_data = Tx_data.darrays[0].data
    Tx_data = Tx_data[select_ind_L]
    assert Tx_data.shape[0] == 29696
    assert ~np.isnan(Tx_data).any()
    print(Tx_data.shape)

    # read the GC data
    x = data_L[:, g]
    y = -1 * Tx_data
    print(np.corrcoef(x,y)[0,1])
    x = np.array([np.sum(x[atlas_data==(i+1)], axis=0)/len(np.argwhere(atlas_data==(i+1))) for i in range(200)])
    y = np.array([np.sum(y[atlas_data==(i+1)], axis=0)/len(np.argwhere(atlas_data==(i+1))) for i in range(200)])
    print(np.corrcoef(x,y)[0,1])

    # plotting
    clist=['#000C4D', '#00679A', '#FFFFFF', '#007F00', '#005000']
    cm = LinearSegmentedColormap.from_list('chaos',clist)
    sns.set_context("paper", font_scale = 1.8)
    plt.figure(figsize=(5.2, 5), dpi=200)
    f = sns.regplot(x,y,
                    scatter_kws={'s': 2},
                    line_kws={"color": "#B30426"})
    f.scatter(x,y, 
              s=10, 
              c='#9d9d9d',
              # c=x, # same color with the GCs
              cmap=cm)
    if g==0: f.set_xlabel(f'GC1-DV')
    if g==1: f.set_xlabel(f'GC2-RC')
    if g==2: f.set_xlabel(f'GC3-ML')
    f.set_ylabel(f'')
    f.grid(False)
    f=f.get_figure()
    f.tight_layout()
    plt.show()

# GGs

In [ ]:
dirc_L = f'/n02dat01/users/dyli/Atlas/metric_index_L.txt'
select_ind_L = np.loadtxt( dirc_L ).astype(int)
dirc_R = f'/n02dat01/users/dyli/Atlas/metric_index_R.txt'
select_ind_R = np.loadtxt( dirc_R ).astype(int)

# read the Yeo Atlas
atlas_path = '/n02dat01/users/dyli/Atlas/Schaefer2018_400Parcels_17Networks_order.dlabel.nii'
atlas = nib.load(atlas_path)
atlas_data_ = atlas.get_fdata()
atlas_data_ = np.squeeze(atlas_data_) #(64984,)
atlas_data = np.zeros(59412)
atlas_data[0:29696] = atlas_data_[select_ind_L]
atlas_data[29696:59412] = atlas_data_[select_ind_R+32492]

In [ ]:
atlas_data = atlas_data[0:29696]
for _f in ['thickness']:
    twin_father = f'/n02dat01/users/dyli/Grad_data/GCM/{_f}/gii'

    for g in range(3):
        GG_L = surface.load_surf_data(f'{twin_father}/dm_Gradient{g}_ACE_new_nomw.fsLR_32k.l.func.gii')
        GG_R = surface.load_surf_data(f'{twin_father}/dm_Gradient{g}_ACE_new_nomw.fsLR_32k.r.func.gii')

        # read the GC data
        if g==2: x = data_L[:, 0]
        if g==0: x = data_L[:, 1]
        if g==1: x = data_L[:, 2]
        y = GG_L[select_ind_L]
        print(np.corrcoef(x,y)[0,1])
        x = np.array([np.sum(x[atlas_data==(i+1)], axis=0)/len(np.argwhere(atlas_data==(i+1))) for i in range(200)])
        y = np.array([np.sum(y[atlas_data==(i+1)], axis=0)/len(np.argwhere(atlas_data==(i+1))) for i in range(200)])
        print(np.corrcoef(x,y)[0,1])

        # plotting
        clist=['#000C4D', '#00679A', '#FFFFFF', '#007F00', '#005000']
        cm = LinearSegmentedColormap.from_list('chaos',clist)
        sns.set_context("paper", font_scale = 1.8)
        plt.figure(figsize=(5.2, 5), dpi=200)
        f = sns.regplot(x,y,
                        scatter_kws={'s': 2},
                        line_kws={"color": "#B30426"})
        f.scatter(x,y, 
                s=10, 
                c='#9d9d9d',
                # c=x,
                cmap=cm)
        if g==2: f.set_xlabel(f'GC1-DV')
        if g==0: f.set_xlabel(f'GC2-RC')
        if g==1: f.set_xlabel(f'GC3-ML')
        x_major_locator=MultipleLocator(5)
        y_major_locator=MultipleLocator(0.1)
        ax=plt.gca()
        ax.xaxis.set_major_locator(x_major_locator)
        ax.yaxis.set_major_locator(y_major_locator)
        f.set_ylabel(f'')
        f.grid(False)
        f=f.get_figure()
        f.tight_layout()
        plt.show()
